<a href="https://colab.research.google.com/github/Nuwantha97/Sinhala_spell_and_grammer_checker/blob/Notebooks/Context_Awareness_with_spell_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# prompt: mount to google drive

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base")

# prepare input
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')

# forward pass
output = model(**encoded_input)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
print(encoded_input)

{'input_ids': tensor([[    0,   853, 23935,   163,   390,  2499,  7986,   398,    25,    71,
          1884,     5,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='xlm-roberta-base')
unmasker("මම <mask> යනවා බස් එකේ.")

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


[{'score': 0.14152264595031738,
  'token': 27737,
  'token_str': 'ඉතින්',
  'sequence': 'මම ඉතින් යනවා බස් එකේ.'},
 {'score': 0.13114693760871887,
  'token': 12943,
  'token_str': 'දැන්',
  'sequence': 'මම දැන් යනවා බස් එකේ.'},
 {'score': 0.04833247512578964,
  'token': 38093,
  'token_str': 'ගෙදර',
  'sequence': 'මම ගෙදර යනවා බස් එකේ.'},
 {'score': 0.04498281702399254,
  'token': 156766,
  'token_str': 'හැමදාම',
  'sequence': 'මම හැමදාම යනවා බස් එකේ.'},
 {'score': 0.03954967111349106,
  'token': 198663,
  'token_str': 'ආපහු',
  'sequence': 'මම ආපහු යනවා බස් එකේ.'}]

In [20]:
import sentencepiece as spm
from transformers import AutoTokenizer
import re

class SinhalaTokenizer:
    def __init__(self):
        # Load SinhalaBERTo tokenizer
        try:
            self.bert_tokenizer = AutoTokenizer.from_pretrained("keshan/SinhalaBERTo") #tokenizer = AutoTokenizer.from_pretrained("keshan/SinhalaBERTo")
        except:
            self.bert_tokenizer = None
            print("SinhalaBERTo tokenizer not available. Using basic tokenization.")

    def basic_tokenize(self, text):
        """Simple rule-based tokenization for Sinhala"""
        # Remove punctuation
        text = re.sub(r'[.,!?]', ' ', text)
        # Split on spaces
        tokens = text.split()
        return tokens

    def bert_tokenize(self, text):
        """Using SinhalaBERTo tokenizer"""
        if self.bert_tokenizer:
            return self.bert_tokenizer.tokenize(text)bert_tokenizer.tokenize(text)
        return None

    def train_sentencepiece(self, input_file, vocab_size=8000):
        """Train a SentencePiece model for Sinhala"""
        spm.SentencePieceTrainer.train(
            f'--input={input_file} --model_prefix=sinhala_sp '
            f'--vocab_size={vocab_size} --character_coverage=0.9995 '
            '--model_type=unigram --input_sentence_size=1000000'
        )
        self.sp = spm.SentencePieceProcessor()
        self.sp.load('sinhala_sp.model')

    def sentencepiece_tokenize(self, text):
        """Tokenize using trained SentencePiece model"""
        if hasattr(self, 'sp'):
            return self.sp.encode_as_pieces(text)
        return None

# Usage example
tokenizer = SinhalaTokenizer()

# Example Sinhala text
text = "මම පරිගණක විද්‍යාව ඉගෙනගන්නවා"

# Basic tokenization
basic_tokens = tokenizer.basic_tokenize(text)

# BERT tokenization
bert_tokens = tokenizer.bert_tokenize(text)

# Train and use SentencePiece (requires training data)
# tokenizer.train_sentencepiece("sinhala_text.txt")
# sp_tokens = tokenizer.sentencepiece_tokenize(text)

In [4]:
print(basic_tokens)

['මම', 'පරිගණක', 'විද්\u200dයාව', 'ඉගෙනගන්නවා']


In [19]:
bert_tokens

['à¶¸à¶¸',
 'Ġà¶´à¶»',
 'à·Ĵ',
 'à¶ľà¶«à¶ļ',
 'Ġà·Ģ',
 'à·Ĵ',
 'à¶¯',
 'à·ĬâĢį',
 'à¶º',
 'à·ı',
 'à·Ģ',
 'Ġà¶īà¶ľ',
 'à·Ļ',
 'à¶±à¶ľà¶±',
 'à·Ĭ',
 'à¶±à·Ģ',
 'à·ı']

In [23]:
# prompt: how to decode the given tokens in 'bert_tokens'

# Decode BERT tokens
decoded_bert_tokens = tokenizer.bert_tokenizer.convert_tokens_to_string(['à·Ĵ',
 'à¶¯',
 'à·ĬâĢį',
 'à¶º',
 'à·ı',
 'à·Ģ',
 'Ġà¶īà¶ľ',
 'à·Ļ'])
decoded_bert_tokens

'ිද්\u200dයාව ඉගෙ'

In [26]:
# Example Sinhala text
sentence = "මම බස් එකේ පස්ලට යනවා"


In [27]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("keshan/SinhalaBERTo")
inputs = tokenizer(sentence, return_tensors="pt")
input_ids = inputs["input_ids"]
attention_mask = inputs["attention_mask"]


In [28]:
inputs

{'input_ids': tensor([[  0, 818, 827, 264, 330, 277, 396, 264, 634, 920, 268,   2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [29]:
input_ids

tensor([[  0, 818, 827, 264, 330, 277, 396, 264, 634, 920, 268,   2]])

In [30]:
# prompt: how to decode 'input_ids'

# Decode input_ids
decoded_sentence = tokenizer.decode(input_ids[0])
decoded_sentence

'<s>මම බස් එකේ පස්ලට යනවා</s>'

# Distilbert

In [35]:
!pip install flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 29.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for flash-attn: filename=flash_attn-2.7.2.post1-cp310-cp310-linux_x86_64.whl size=190160474 sha256=0b454d9e650bfc437cc71335080172a5d05f51eab355636c9d5b7321fec7318e
  Stored in directory: /root/.cache/pip/wheels/da/ec/5b/b2c37a8e4f755ad82492a822463bca0817f0e0e11de874b550
Successfully built flash-attn


In [36]:
import torch
from transformers import AutoTokenizer, AutoModel

device = "cuda" # the device to load the model onto

tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased')
model = AutoModel.from_pretrained("distilbert/distilbert-base-uncased", torch_dtype=torch.float16, attn_implementation="flash_attention_2")

text = "මම බස් එකේ පස්ලට යනවා."

encoded_input = tokenizer(text, return_tensors='pt').to(device)
model.to(device)

output = model(**encoded_input)

ValueError: FlashAttention2 has been toggled on, but it cannot be used due to the following error: Flash Attention 2 is not available on CPU. Please make sure torch can access a CUDA device.

In [37]:
from transformers import AutoTokenizer, DistilBertModel
import torch

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [43]:
from transformers import DistilBertTokenizer, DistilBertForMaskedLM
import torch

def predict_masked_word(text, mask_token="[MASK]"):
    # Initialize tokenizer and model
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    model = DistilBertForMaskedLM.from_pretrained('distilbert-base-uncased')

    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt")

    # Get the position of the masked token
    mask_token_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)[0]

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get probabilities for all tokens
    predictions = outputs.logits[0, mask_token_index]
    top_tokens = torch.topk(predictions, k=5, dim=1)

    # Convert token IDs to words
    for scores, indices in zip(top_tokens.values, top_tokens.indices):
        predicted_tokens = [tokenizer.decode([idx]) for idx in indices]
        probabilities = torch.softmax(scores, dim=0)

        # Return results as list of tuples (word, probability)
        return list(zip(predicted_tokens, probabilities.tolist()))

# Example usage
text = "The cat [MASK] on the mat."
predictions = predict_masked_word(text)

for word, prob in predictions[0]:
    print(f"Word: {word.strip()}, Probability: {prob:.3f}")

ValueError: too many values to unpack (expected 2)

In [59]:
from transformers import DistilBertTokenizer, DistilBertForMaskedLM
import torch

def predict_masked_word(text, mask_token="[MASK]"):
    # Initialize tokenizer and model
    tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
    model = DistilBertForMaskedLM.from_pretrained('distilbert-base-uncased')

    # Tokenize input text
    inputs = tokenizer(text, return_tensors="pt")

    # Get the position of the masked token
    mask_token_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)[0]

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get probabilities for all tokens
    predictions = outputs.logits[0, mask_token_index]
    top_tokens = torch.topk(predictions, k=5, dim=1)

    # Convert token IDs to words
    for scores, indices in zip(top_tokens.values, top_tokens.indices):
        predicted_tokens = [tokenizer.decode([idx]) for idx in indices]
        probabilities = torch.softmax(scores, dim=0)

        # Return results as list of tuples (word, probability)
        return list(zip(predicted_tokens, probabilities.tolist()))

# Example usage
#text = "The cat [MASK] on the mat."
text = "මම බස් එකේ [MASK] යනවා."
predictions = predict_masked_word(text)

# Iterate through the list of predictions directly
# Each prediction is a tuple of (word, probability)
for word, prob in predictions:  # Removed [0] for proper iteration
    print(f"Word: {word.strip()}, Probability: {prob:.3f}")

Word: 的, Probability: 0.252
Word: 不, Probability: 0.229
Word: ，, Probability: 0.226
Word: [UNK], Probability: 0.176
Word: 之, Probability: 0.118


In [ ]:
def sinhala_to_unicode(text):
    """
    Convert Sinhala text to proper Unicode representation.

    Args:
        text (str): Input Sinhala text

    Returns:
        str: Unicode converted text
    """
    # Mapping dictionary for Sinhala characters
    sinhala_mapping = {
        'අ': '\u0D85',    # ayanna
        'ආ': '\u0D86',    # aayanna
        'ඇ': '\u0D87',    # aeyanna
        'ඈ': '\u0D88',    # aeeyanna
        'ඉ': '\u0D89',    # iyanna
        'ඊ': '\u0D8A',    # iiyanna
        'උ': '\u0D8B',    # uyanna
        'ඌ': '\u0D8C',    # uuyanna
        'ඍ': '\u0D8D',    # iruyanna
        'ඎ': '\u0D8E',    # iruuyanna
        'ඏ': '\u0D8F',    # iluyanna
        'ඐ': '\u0D90',    # iluuyanna
        'එ': '\u0D91',    # eyanna
        'ඒ': '\u0D92',    # eeyanna
        'ඓ': '\u0D93',    # aiyanna
        'ඔ': '\u0D94',    # oyanna
        'ඕ': '\u0D95',    # ooyanna
        'ඖ': '\u0D96',    # auyanna

        # Consonants
        'ක': '\u0D9A',    # kayanna
        'ඛ': '\u0D9B',    # maha kayanna
        'ග': '\u0D9C',    # gayanna
        'ඝ': '\u0D9D',    # maha gayanna
        'ඞ': '\u0D9E',    # kantaja naasikyaya
        'ඟ': '\u0D9F',    # sanyaka gayanna
        'ච': '\u0DA0',    # cayanna
        'ඡ': '\u0DA1',    # maha cayanna
        'ජ': '\u0DA2',    # jayanna
        'ඣ': '\u0DA3',    # maha jayanna
        'ඤ': '\u0DA4',    # taaluja naasikyaya
        'ඥ': '\u0DA5',    # taaluja sanyooga naasikyaya

        # Modifiers
        '්': '\u0DCA',    # hal kirima
        'ා': '\u0DCF',    # aela-pilla
        'ැ': '\u0DD0',    # ketti aeda-pilla
        'ෑ': '\u0DD1',    # diga aeda-pilla
        'ි': '\u0DD2',    # ketti is-pilla
        'ී': '\u0DD3',    # diga is-pilla
        'ු': '\u0DD4',    # ketti paa-pilla
        'ූ': '\u0DD6',    # diga paa-pilla
    }

    result = ''
    for char in text:
        # If character exists in mapping, convert it, otherwise keep original
        result += sinhala_mapping.get(char, char)

    return result

# Example usage
def test_converter():
    # Test cases
    test_texts = [
        "අම්මා",      # amma (mother)
        "තාත්තා",    # thaththa (father)
        "මල්ලි",      # malli (younger brother)
        "අක්කා"      # akka (elder sister)
    ]

    print("Testing Sinhala Unicode conversion:")
    for text in test_texts:
        unicode_text = sinhala_to_unicode(text)
        print(f"Original: {text}")
        print(f"Unicode: {unicode_text}")
        print(f"Unicode points: {[hex(ord(c)) for c in unicode_text]}")
        print()

if __name__ == "__main__":
    test_converter()

# Using xlm-roberta-base

In [57]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import re

def is_sinhala(text):
    return bool(re.match(r'^[\u0D80-\u0DFF]+$', text.strip()))

def predict_masked_sinhala(text, k=20):
    model_name = "xlm-roberta-base"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForMaskedLM.from_pretrained(model_name)

    text = text.replace("[MASK]", tokenizer.mask_token)
    inputs = tokenizer(text, return_tensors="pt")
    mask_token_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)[0]

    with torch.no_grad():
        outputs = model(**inputs)

    predictions = outputs.logits[0, mask_token_index]
    top_tokens = torch.topk(predictions, k=k, dim=1)

    results = []
    for scores, indices in zip(top_tokens.values, top_tokens.indices):
        predicted_tokens = [tokenizer.decode([idx]) for idx in indices]
        probabilities = torch.softmax(scores, dim=0)

        # Filter Sinhala predictions
        sinhala_results = [(word.strip(), prob)
                          for word, prob in zip(predicted_tokens, probabilities.tolist())
                          if any(is_sinhala(char) for char in word)]
        results.extend(sinhala_results)

    return results[:10]  # Return top 10 Sinhala predictions

# Example usage
text = "මම [MASK] යනවා."
predictions = predict_masked_sinhala(text)

print("Top Sinhala Predictions:")
for i, (word, prob) in enumerate(predictions, 1):
    print(f"{i}. Word: {word}, Probability: {prob:.3f}")

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Top Sinhala Predictions:
1. Word: යන්න, Probability: 0.029
2. Word: ගෙදර, Probability: 0.027
3. Word: එළියට, Probability: 0.015


In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForMaskedLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments
)
from datasets import Dataset
import torch

def prepare_data(texts):
    return Dataset.from_dict({"text": texts})

def tokenize_function(examples):
    tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
    return tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

def train_model(train_texts, eval_texts, output_dir="./finetuned-xlm-roberta"):
    # Initialize tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
    model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base")

    # Prepare datasets
    train_dataset = prepare_data(train_texts)
    eval_dataset = prepare_data(eval_texts)

    # Tokenize datasets
    tokenized_train = train_dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=train_dataset.column_names
    )
    tokenized_eval = eval_dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=eval_dataset.column_names
    )

    # Data collator
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=True,
        mlm_probability=0.15
    )

    # Training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=3,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        learning_rate=2e-5,
        weight_decay=0.01,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_dir="./logs",
    )

    # Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_eval,
        data_collator=data_collator,
    )

    # Train
    trainer.train()

    # Save model
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

    return model, tokenizer

# Example usage
train_texts = [
    "මම ගෙදර යනවා.",
    "ඔහු පාසල් යනවා.",
    # Add more training examples
]

eval_texts = [
    "අපි කඩේට යනවා.",
    "ඇය රට යනවා.",
    # Add more evaluation examples
]

model, tokenizer = train_model(train_texts, eval_texts)

# Indic bert

In [61]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import re

def is_sinhala(text):
    return bool(re.match(r'^[\u0D80-\u0DFF]+$', text.strip()))

def predict_masked_sinhala(text, mask_token="[MASK]", min_results=5):
    model_name = "ai4bharat/indic-bert"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForMaskedLM.from_pretrained(model_name)

    text = text.replace("[MASK]", tokenizer.mask_token)
    inputs = tokenizer(text, return_tensors="pt")
    mask_token_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)[0]

    with torch.no_grad():
        outputs = model(**inputs)

    predictions = outputs.logits[0, mask_token_index]

    # Keep getting more predictions until we have enough Sinhala results
    k = 50  # Start with 50 predictions
    results = []
    while len(results) < min_results and k <= 500:  # Cap at 500 to prevent infinite loop
        top_tokens = torch.topk(predictions, k=k, dim=1)
        predicted_tokens = [tokenizer.decode([idx]) for idx in top_tokens.indices[0]]
        probabilities = torch.softmax(top_tokens.values[0], dim=0)

        results = [(word, prob) for word, prob in zip(predicted_tokens, probabilities.tolist())
                  if is_sinhala(word)]
        k *= 2  # Double k if we need more results

    return results[:max(min_results, len(results))]

# Example usage
text = "මම [MASK] කරන්න යනවා."
predictions = predict_masked_sinhala(text)

for word, prob in predictions:
    print(f"Word: {word.strip()}, Probability: {prob:.3f}")

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForMaskedLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from datasets import Dataset
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def prepare_sinhala_dataset(file_path):
    """
    Prepare dataset from a text file or CSV containing Sinhala text
    """
    # Assuming your dataset is in CSV format with a 'text' column
    # Modify this according to your data format
    df = pd.read_csv(file_path)

    train_df, eval_df = train_test_split(df, test_size=0.1, random_state=42)

    train_dataset = Dataset.from_pandas(train_df)
    eval_dataset = Dataset.from_pandas(eval_df)

    return train_dataset, eval_dataset

def tokenize_function(examples, tokenizer):
    """
    Tokenize the text examples
    """
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128,
        return_special_tokens_mask=True
    )

def fine_tune_model(
    train_dataset,
    eval_dataset,
    model_name="ai4bharat/indic-bert",
    output_dir="./sinhala-bert-finetuned",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=2e-5
):
    """
    Fine-tune the model using the Trainer
    """
    # Initialize tokenizer and model
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForMaskedLM.from_pretrained(model_name)

    # Tokenize datasets
    tokenized_train = train_dataset.map(
        lambda x: tokenize_function(x, tokenizer),
        batched=True,
        remove_columns=train_dataset.column_names
    )
    tokenized_eval = eval_dataset.map(
        lambda x: tokenize_function(x, tokenizer),
        batched=True,
        remove_columns=eval_dataset.column_names
    )

    # Data collator
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=True,
        mlm_probability=0.15
    )

    # Training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=per_device_train_batch_size,
        per_device_eval_batch_size=per_device_train_batch_size,
        gradient_accumulation_steps=gradient_accumulation_steps,
        evaluation_strategy="steps",
        eval_steps=500,
        save_steps=500,
        save_total_limit=2,
        learning_rate=learning_rate,
        weight_decay=0.01,
        warmup_steps=500,
        logging_dir="./logs",
        logging_steps=100,
        load_best_model_at_end=True,
        metric_for_best_model="loss",
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_eval,
        data_collator=data_collator,
    )

    # Train the model
    trainer.train()

    # Save the final model
    trainer.save_model()
    tokenizer.save_pretrained(output_dir)

    return model, tokenizer

def main():
    # Path to your Sinhala dataset
    dataset_path = "path_to_your_sinhala_dataset.csv"

    # Prepare datasets
    train_dataset, eval_dataset = prepare_sinhala_dataset(dataset_path)

    # Fine-tune the model
    model, tokenizer = fine_tune_model(
        train_dataset,
        eval_dataset,
        output_dir="./sinhala-bert-finetuned",
        num_train_epochs=3
    )

    # Test the fine-tuned model
    test_text = "මම [MASK] කරන්න යනවා."
    inputs = tokenizer(test_text, return_tensors="pt")
    mask_token_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)[0]

    with torch.no_grad():
        outputs = model(**inputs)
        predictions = outputs.logits[0, mask_token_index]
        top_tokens = torch.topk(predictions, k=5, dim=1)

    for token, score in zip(top_tokens.indices[0], top_tokens.values[0]):
        word = tokenizer.decode([token])
        print(f"Predicted word: {word}, Score: {score:.3f}")

if __name__ == "__main__":
    main()

# DistilBERT

In [2]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from datasets import Dataset
from datasets import load_dataset

sold_train = Dataset.to_pandas(load_dataset('sinhala-nlp/SOLD', split='train'))[['text']]
sold_test = Dataset.to_pandas(load_dataset('sinhala-nlp/SOLD', split='test'))[['text']]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.93k [00:00<?, ?B/s]

SOLD_train.tsv:   0%|          | 0.00/4.72M [00:00<?, ?B/s]

SOLD_test.tsv:   0%|          | 0.00/1.60M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2500 [00:00<?, ? examples/s]

In [4]:
# prompt: all sold_train and sold_test data include to one dataframe call data
import pandas as pd

data = pd.concat([sold_train, sold_test], ignore_index=True)

In [5]:
import re
import unicodedata

def clean_sinhala_text(text):
    """
    Clean Sinhala text while preserving meaningful special characters
    """
    # Characters to keep
    sinhala_marks = {
        '.',   # Full stop
        '।',   # Sinhala full stop
        '෴',   # Sinhala punctuation kunddaliya
        '॥',   # Double danda
        '?',   # Question mark
        '!',   # Exclamation mark
        ',',   # Comma
        ';',   # Semicolon
        '"',   # Quotation marks
        '"',   # Sinhala quotation mark
        '"',   # Sinhala quotation mark
         "'"  # Apostrophe
    }

    # Step 1: Normalize Unicode characters
    text = unicodedata.normalize('NFKC', text)

    # Step 2: Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Step 3: Remove HTML tags
    text = re.sub(r'<.*?>', '', text)

    # Step 4: Remove emojis and other symbols
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)

    # Step 5: Keep only Sinhala characters and allowed special characters
    def keep_char(char):
        # Keep Sinhala Unicode range
        if '\u0D80' <= char <= '\u0DFF':
            return True
        # Keep specific punctuation
        if char in sinhala_marks:
            return True
        # Keep spaces and newlines
        if char.isspace():
            return True
        return False

    text = ''.join(char for char in text if keep_char(char))

    # Step 6: Clean up extra whitespace
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()

    return text

def process_dataset(texts):
    """
    Process a list of texts and provide statistics
    """
    processed_texts = []
    stats = {
        'original_count': len(texts),
        'processed_count': 0,
        'empty_after_cleaning': 0,
        'special_chars_found': set()
    }

    for text in texts:
        cleaned = clean_sinhala_text(text)
        if cleaned:
            processed_texts.append(cleaned)
            # Collect statistics about special characters
            special_chars = set(char for char in cleaned
                              if not '\u0D80' <= char <= '\u0DFF'
                              and not char.isspace())
            stats['special_chars_found'].update(special_chars)
        else:
            stats['empty_after_cleaning'] += 1

    stats['processed_count'] = len(processed_texts)
    return processed_texts, stats


In [6]:
preProcessed_data = data['text'].apply(clean_sinhala_text)

In [7]:
preProcessed_data

,text
0,පට්ට පට පට...
1,පරණ කෑල්ල අද වෙනකම් හිටියනම් අදට අවුරුදු යි. ය...
2,යාළුවා කියලා හිතන් සර් ගේ ඔලුවට රෙද්ද දාලා නෙල...
3,හොඳ මිතුරියක් කතා කලා. විස්තර කතාකරමින් ඉදලා ම...
4,"ඔය බනින්නෙ.. හරකා, මී හරකා කිය කිය..."
...,...
9995,"පට්ට වේසි, දැන් එනව මෙතන රහත් වෙන්න"
9996,"මේක හරි හුත්තක් උනානේ ""කොල්ලෙක් නෑ ,කෙල්ලෙක් න..."
9997,ලොකු නංඟි ගෙ පරණ කොල්ලා ඒකිට වද දෙනවලු ඒ වුනාට...
9998,බය කරන්න එපා යකෝ සිත්තරා මේය ලොව ඇති භයානකම චි...


In [8]:
# prompt: preProcessed_train and preProcessed_test save as csv files

preProcessed_data.to_csv('preProcessed_data.csv', index=False)

In [9]:
from transformers import DistilBertTokenizer, DistilBertForMaskedLM
import torch
import re

def is_sinhala(text):
    return bool(re.match(r'^[\u0D80-\u0DFF]+$', text.strip()))

def predict_masked_sinhala(text, k=20):
    # Initialize DistilBERT tokenizer and model
    model_name = "distilbert-base-multilingual-cased"  # Using multilingual DistilBERT
    tokenizer = DistilBertTokenizer.from_pretrained(model_name)
    model = DistilBertForMaskedLM.from_pretrained(model_name)

    # Replace [MASK] with the model's mask token
    text = text.replace("[MASK]", tokenizer.mask_token)

    # Tokenize input
    inputs = tokenizer(text, return_tensors="pt")

    # Find position of mask token
    mask_token_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)[0]

    # Generate predictions
    with torch.no_grad():
        outputs = model(**inputs)

    # Get predictions for masked token
    predictions = outputs.logits[0, mask_token_index]
    top_tokens = torch.topk(predictions, k=k, dim=1)

    results = []
    for scores, indices in zip(top_tokens.values, top_tokens.indices):
        predicted_tokens = [tokenizer.decode([idx]) for idx in indices]
        probabilities = torch.softmax(scores, dim=0)

        # Filter Sinhala predictions
        sinhala_results = [(word.strip(), prob)
                          for word, prob in zip(predicted_tokens, probabilities.tolist())
                          if any(is_sinhala(char) for char in word)]
        results.extend(sinhala_results)

    return results[:10]  # Return top 10 Sinhala predictions

# Example usage
if __name__ == "__main__":
    text = "මම [MASK] යනවා."
    predictions = predict_masked_sinhala(text)
    print("Top Sinhala Predictions:")
    for i, (word, prob) in enumerate(predictions, 1):
        print(f"{i}. Word: {word}, Probability: {prob:.3f}")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Top Sinhala Predictions:


In [ ]:
from transformers import (
    DistilBertTokenizer,
    DistilBertForMaskedLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments
)
from datasets import Dataset
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def prepare_dataset(file_path):
    """
    Prepare dataset from a text file or DataFrame containing Sinhala text
    Args:
        file_path: Path to your Sinhala dataset (CSV, TXT, etc.)
    """
    # This is an example - modify according to your data format
    try:
        # For CSV files
        df = pd.read_csv(file_path)
        texts = df['text'].tolist()  # Assuming 'text' is your column name
    except:
        # For text files
        with open(file_path, 'r', encoding='utf-8') as f:
            texts = f.readlines()

    return Dataset.from_dict({'text': texts})

def tokenize_function(examples, tokenizer):
    """Tokenize the text and create input_ids and attention_mask"""
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=128,
        return_special_tokens_mask=True
    )

def main():
    # Initialize model and tokenizer
    model_name = "distilbert-base-multilingual-cased"
    tokenizer = DistilBertTokenizer.from_pretrained(model_name)
    model = DistilBertForMaskedLM.from_pretrained(model_name)

    # Prepare dataset
    dataset = prepare_dataset("/content/preProcessed_data.csv")  # Replace with your dataset path

    # Convert Dataset to pandas DataFrame for splitting
    df = dataset.to_pandas()

    # Split dataset using pandas DataFrame
    train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
    valid_df, test_df = train_test_split(test_df, test_size=0.5, random_state=42)

    # Convert back to Hugging Face Datasets
    train_dataset = Dataset.from_pandas(train_df)
    valid_dataset = Dataset.from_pandas(valid_df)
    test_dataset = Dataset.from_pandas(test_df)


    # Tokenize datasets
    tokenized_train = train_dataset.map(
        lambda x: tokenize_function(x, tokenizer),
        batched=True,
        remove_columns=train_dataset.column_names
    )
    tokenized_valid = valid_dataset.map(
        lambda x: tokenize_function(x, tokenizer),
        batched=True,
        remove_columns=valid_dataset.column_names
    )

    # Create data collator
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=True,
        mlm_probability=0.15
    )

    # Define training arguments
    training_args = TrainingArguments(
        output_dir="./sinhala_distilbert",
        overwrite_output_dir=True,
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        evaluation_strategy="steps",
        eval_steps=500,
        save_steps=500,
        save_total_limit=2,
        logging_dir='./logs',
        logging_steps=100,
        learning_rate=2e-5,
        weight_decay=0.01,
        warmup_steps=500,
        load_best_model_at_end=True,
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_valid,
        data_collator=data_collator,
    )

    # Train the model
    trainer.train()

    # Save the model
    model.save_pretrained("./sinhala_distilbert_final")
    tokenizer.save_pretrained("./sinhala_distilbert_final")

    # Evaluate on test set
    eval_results = trainer.evaluate(test_dataset)
    print(f"Test set results: {eval_results}")

if __name__ == "__main__":
    main()

Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [10]:
from transformers import (
    DistilBertTokenizer,
    DistilBertForMaskedLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments
)
from datasets import Dataset
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def prepare_dataset(file_path):
    """
    Prepare dataset from a text file or DataFrame containing Sinhala text
    Args:
        file_path: Path to your Sinhala dataset (CSV, TXT, etc.)
    """
    try:
        df = pd.read_csv(file_path)
        texts = df['text'].tolist()
    except:
        with open(file_path, 'r', encoding='utf-8') as f:
            texts = f.readlines()

    # Remove empty strings and None values
    texts = [str(text).strip() for text in texts if str(text).strip()]
    return Dataset.from_dict({'text': texts})

def tokenize_function(examples, tokenizer):
    """Tokenize the text and create input_ids and attention_mask"""
    result = tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=128,
        return_special_tokens_mask=True
    )
    return result

def main():
    # Initialize model and tokenizer
    model_name = "distilbert-base-multilingual-cased"
    tokenizer = DistilBertTokenizer.from_pretrained(model_name)
    model = DistilBertForMaskedLM.from_pretrained(model_name)

    # Prepare dataset
    dataset = prepare_dataset("/content/preProcessed_data.csv")

    # Split dataset
    train_test = dataset.train_test_split(test_size=0.2, seed=42)
    test_valid = train_test['test'].train_test_split(test_size=0.5, seed=42)

    train_dataset = train_test['train']
    valid_dataset = test_valid['train']
    test_dataset = test_valid['test']

    # Tokenize datasets
    tokenized_train = train_dataset.map(
        lambda x: tokenize_function(x, tokenizer),
        batched=True,
        remove_columns=train_dataset.column_names,
        desc="Tokenizing train dataset"
    )

    tokenized_valid = valid_dataset.map(
        lambda x: tokenize_function(x, tokenizer),
        batched=True,
        remove_columns=valid_dataset.column_names,
        desc="Tokenizing validation dataset"
    )

    tokenized_test = test_dataset.map(
        lambda x: tokenize_function(x, tokenizer),
        batched=True,
        remove_columns=test_dataset.column_names,
        desc="Tokenizing test dataset"
    )

    # Create data collator
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=True,
        mlm_probability=0.15
    )

    # Define training arguments
    training_args = TrainingArguments(
        output_dir="./sinhala_distilbert",
        overwrite_output_dir=True,
        num_train_epochs=3,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        eval_strategy="steps",  # Updated from evaluation_strategy
        eval_steps=500,
        save_steps=500,
        save_total_limit=2,
        logging_dir='./logs',
        logging_steps=100,
        learning_rate=2e-5,
        weight_decay=0.01,
        warmup_steps=200,
        load_best_model_at_end=True,
        report_to="wandb"  # Add this if you want to use wandb
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_valid,
        data_collator=data_collator,
    )

    # Train the model
    try:
        trainer.train()
    except Exception as e:
        print(f"Training error: {e}")
        return

    # Save the model
    try:
        model.save_pretrained("./sinhala_distilbert_final")
        tokenizer.save_pretrained("./sinhala_distilbert_final")
    except Exception as e:
        print(f"Saving error: {e}")
        return

    # Evaluate on test set
    try:
        eval_results = trainer.evaluate(tokenized_test)
        print(f"Test set results: {eval_results}")
    except Exception as e:
        print(f"Evaluation error: {e}")

if __name__ == "__main__":
    main()

Tokenizing train dataset:   0%|          | 0/8000 [00:00<?, ? examples/s]

Tokenizing validation dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing test dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [12]:
# After training
model.save_pretrained("./content/drive/MyDrive/Projects/DistilBERT_model")
tokenizer.save_pretrained("./content/drive/MyDrive/Projects/DistilBERT(Tok)_model")

NameError: name 'model' is not defined